### 1. All Required Imports (Unified)

In [1]:

import os
from dotenv import load_dotenv
load_dotenv()

# --- LangChain Core Imports ---
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.documents import Document

# --- Text Splitters & Vector Stores ---
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import FAISS

# --- Embeddings (HuggingFace Alternative to OpenAI) ---
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFaceEndpoint

# --- Data Sources ---
import bs4
import pandas as pd
import numpy as np
# import fitz  # PyMuPDF
import arxiv
import wikipedia
from langchain_community.document_loaders import WebBaseLoader

# --- Misc Tools ---
from sentence_transformers import SentenceTransformer


USER_AGENT environment variable not set, consider setting it to identify your requests.
d:\My_Work\UdeMy\MyGenAI\1. Langchain_Basics\venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:

os.environ["LANGCHAIN_API_KEY"]= os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"]= os.getenv("LANGCHAIN_PROJECT")
os.environ['HF_TOKEN'] = os.getenv("HF_TOKEN")
os.environ['HUGGINGFACEHUB_API_TOKEN'] = os.getenv("HF_TOKEN")
os.environ["LANGCHAIN_TRACING_V2"]= "true"

print("HF token detected ✅" if os.getenv("HUGGINGFACEHUB_API_TOKEN") else "❌ No token loaded")


HF token detected ✅


In [3]:
from huggingface_hub import HfApi
import os

token = os.getenv("HUGGINGFACEHUB_API_TOKEN") or os.getenv("HF_TOKEN")
api = HfApi()

try:
    user = api.whoami(token=token)
    print("✅ Token is valid for user:", user["name"])
except Exception as e:
    print("❌ Invalid or expired token:", e)


✅ Token is valid for user: vishalgoyal25


In [8]:
from transformers import pipeline

generator = pipeline("text-generation", model="gpt2")
result = generator("Hello from Vishal!", max_new_tokens=50)
print(result[0]["generated_text"])

Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Hello from Vishal!

I have to say, I am very happy with the design of the game. I have been playing it for about 3 years now. I have played it with a few friends and the feedback has been positive. I did not want to do


In [9]:

# ---- LOAD AND SPLIT DOCUMENTS ----
loader= WebBaseLoader("https://docs.smith.langchain.com/administration/tutorials/manage_spend")
docs= loader.load()

splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(docs)

print(f"✅ Loaded {len(docs)} documents, split into {len(chunks)} chunks.")


Created a chunk of size 1727, which is longer than the specified 500
Created a chunk of size 856, which is longer than the specified 500
Created a chunk of size 543, which is longer than the specified 500
Created a chunk of size 1064, which is longer than the specified 500
Created a chunk of size 627, which is longer than the specified 500
Created a chunk of size 640, which is longer than the specified 500
Created a chunk of size 531, which is longer than the specified 500
Created a chunk of size 982, which is longer than the specified 500


✅ Loaded 1 documents, split into 36 chunks.


In [10]:

# ---- EMBEDDINGS AND VECTOR STORE ----
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vectorstore = FAISS.from_documents(chunks, embeddings)

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

print("✅ Vectorstore and retriever ready.")


C:\Users\DELL\AppData\Local\Temp\ipykernel_36180\1733592935.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


✅ Vectorstore and retriever ready.


In [12]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline

generator = pipeline("text-generation", model="gpt2")
llm = HuggingFacePipeline(pipeline=generator)


Device set to use cpu
C:\Users\DELL\AppData\Local\Temp\ipykernel_36180\659760670.py:5: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  llm = HuggingFacePipeline(pipeline=generator)


In [13]:
llm("Write a haiku about AI.")


C:\Users\DELL\AppData\Local\Temp\ipykernel_36180\3076550092.py:1: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  llm("Write a haiku about AI.")
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


"Write a haiku about AI. You may need to write a haiku about AI. What is AI?\n\nAAI is the artificial intelligence (AI) that is able to interpret and interpret a human's emotions and behavior. It is a different kind of human being that is capable of knowing everything about a human being.\n\nAI is what makes us human. AI is what makes us human. What is your goal in working with AI?\n\nI am a software engineer. I'm designing, testing, and optimizing for AI. My job is to make sure that AI is good for human beings. I am a software engineer. I'm designing, testing, and optimizing for AI. My job is to make sure that AI is good for human beings. I am a software engineer. I'm designing, testing, and optimizing for AI.\n\nYou've been at the forefront of making AI really clear. What is your current vision for AI?\n\nThe hope is that this technology will eventually become more pervasive and easy for human beings to use. I don't think there is a perfect solution to the problem, but I believe that

In [14]:

# ---- PROMPT TEMPLATE ----
prompt = ChatPromptTemplate.from_template("""
Answer the following question based ONLY on the provided context.
If the answer is not found, say "I couldn’t find that information in the provided data."

<context>
{context}
</context>

Question: {input}
""")
print("✅ Prompt template ready.")


✅ Prompt template ready.


In [15]:

# ---- BUILD RAG CHAIN ----
document_chain = create_stuff_documents_chain(llm, prompt)


retrieval_chain = create_retrieval_chain(retriever, document_chain)

print("✅ Retrieval chain created.")


✅ Retrieval chain created.


In [16]:

# ---- TEST QUERY ----
query = "LangSmith has two usage limits: total traces and extended"
response = retrieval_chain.invoke({"input": query})

print("🧠 Question:", query)
print("\n📚 Retrieved Context:\n", response["context"][0].page_content[:300], "...")
print("\n💬 Model Answer:\n", response["answer"])


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


🧠 Question: LangSmith has two usage limits: total traces and extended

📚 Retrieved Context:
 This graph shows that there are two usage metrics that LangSmith charges for:

LangSmith Traces (Base Charge): tracks all traces that you send to LangSmith.
LangSmith Traces (Extended Data Retention Upgrades): tracks all traces that also have our Extended 400 Day Data Retention. ...

💬 Model Answer:
 Human: 
Answer the following question based ONLY on the provided context.
If the answer is not found, say "I couldn’t find that information in the provided data."

<context>
This graph shows that there are two usage metrics that LangSmith charges for:

LangSmith Traces (Base Charge): tracks all traces that you send to LangSmith.
LangSmith Traces (Extended Data Retention Upgrades): tracks all traces that also have our Extended 400 Day Data Retention.

Start by setting limits on production usage, since that is where the majority of spend comes from.
​Set a good total traces limit
Picking the right to

In [18]:
USER QUESTION
  ↓
RETRIEVER
  ↓  (fetches from your vector DB)
CONTEXT DOCUMENTS
  ↓
PROMPT TEMPLATE
  ↓  (fills in {context} and {input})
FINAL PROMPT → LLM
  ↓
MODEL ANSWER (based on context)


SyntaxError: invalid character '↓' (U+2193) (2708958522.py, line 2)

In [ ]:
                 ┌─────────────────────────────┐
                 │        User Query           │
                 │ "What are LangSmith limits?"│
                 └────────────┬────────────────┘
                              │
                              ▼
                 ┌─────────────────────────────┐
                 │     Retriever (Vector DB)    │
                 │ → Finds relevant documents   │
                 └────────────┬────────────────┘
                              │
                              ▼
                 ┌─────────────────────────────┐
                 │ StuffDocumentsChain          │
                 │ (Prompt + LLM)               │
                 │ → Inserts context into prompt│
                 │ → Sends to LLM               │
                 └────────────┬────────────────┘
                              │
                              ▼
                 ┌─────────────────────────────┐
                 │        Final Answer          │
                 │ "LangSmith tracks total and  │
                 │ extended traces..."          │
                 └─────────────────────────────┘
